In [1]:
pal = [(39/255,95/255,153/255),(80/255,176/255,220/255),
       (237/255,244/255,248/255),
       (146/255,144/255,142/255),
       (78/255,78/255,80/255),
(235/255,154/255,88/255),(87/255,126/255,130/255)]

import matplotlib.colors as mcolors
import sklearn.cluster
import numpy as np
import argparse
import cv2

from tqdm import tqdm
import sys
sys.path.append("/Users/rcarlson/Documents/GitHub/lasagna3/snakes")
import mahotas
from ops.imports_ipython import *

import skimage
import umap.umap_ as umap
import stringdb



In [ ]:
thresh = 500
thresh_dofind = 0

loc = "../Data/F1supp/20X_BFP-GFP-Cy5_Slice0/*.tif"
folder = glob(loc)
folder = np.sort(folder)[0:]
corrs0 = []
ind = 2
for i in range(len(folder)):
    site = re.findall("/(.*)_Site-(.*)", (folder)[i])[0][1].split('.')[0]
    #print(site)
   
    data=read(folder[i])
    data[ind][data[ind] < thresh_dofind] = 0
    corrs0.append(np.corrcoef(data[0][data[0] > thresh],data[ind][data[0] > thresh])[1,0])
    
print(np.mean(corrs0))

loc = "../Data/F1supp/20X_BFP-GFP-Cy5_Slice1/*.tif"
folder = glob(loc)
folder = np.sort(folder)[0:]
corrs1 = []
for i in range(len(folder)):
    site = re.findall("/(.*)_Site-(.*)", (folder)[i])[0][1].split('.')[0]
    #print(site)
   
    data=read(folder[i])
    data[ind][data[ind] < thresh_dofind] = 0

    corrs1.append(np.corrcoef(data[0][data[0] > thresh],data[ind][data[0] > thresh])[1,0])

print(np.mean(corrs1))


loc = "../Data/F1supp/20X_BFP-GFP-Cy5_Slice2/*.tif"
folder = glob(loc)
folder = np.sort(folder)[0:]
corrs2 = []
for i in range(len(folder)):
    site = re.findall("/(.*)_Site-(.*)", (folder)[i])[0][1].split('.')[0]
    print(site)
  
    data=read(folder[i])
    data[ind][data[ind] < thresh_dofind] = 0

    corrs2.append(np.corrcoef(data[0][data[0] > thresh],data[ind][data[0] > thresh])[1,0])
    

loc = "../Data/F1supp/20X_BFP-GFP-Cy5_Slice3/*.tif"
folder = glob(loc)
folder = np.sort(folder)[0:]
corrs3 = []
for i in range(len(folder)):
    site = re.findall("/(.*)_Site-(.*)", (folder)[i])[0][1].split('.')[0]
    #print(site)
 
    data=read(folder[i])
    data[ind][data[ind] < thresh_dofind] = 0

    corrs3.append(np.corrcoef(data[0][data[0] > thresh],data[ind][data[0] > thresh])[1,0])
    
print(np.mean(corrs0),np.mean(corrs1),np.mean(corrs2),np.mean(corrs3))